In [2]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from gensim import utils
import gensim.parsing.preprocessing as gsp
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
from sklearn import utils as skl_utils
from tqdm import tqdm
import re
import multiprocessing
import numpy as np

In [3]:
fake_real_data = pd.read_csv("fake_job_postings.csv")

In [4]:
fake_real_data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
fake_real_data.iloc[0,6]

'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staff'

In [6]:
fake_real_data = fake_real_data.dropna()

In [7]:
#Gensim preprocessing filters to remove numberc values, tags, punctuation, multiple whitespaces and stopwords
filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]
#Function to clean the description
def clean_text(data):
    data = data.lower()
    data = utils.to_unicode(data)
    for fil in filters:
        data = fil(data)
    return data

In [8]:
#Example run of clean text
clean_text(fake_real_data.iloc[0,6])

'respons manag english speak editori team build team best class editorsset content creation schedul ensur deadlin adher toresearch write latest tech topic new relat android ecosystemensur content site consist high qualityb face voic url adbddeccedeefeceeaa'

In [9]:
#Split the data into x and Y
train_data = fake_real_data['description']
test_data = fake_real_data['fraudulent']
print("len of df_x",len(train_data))
print("len of df_y",len(test_data))


len of df_x 774
len of df_y 774


In [10]:
#Vectorize the attributes using TF IDF format to weight each word in a document
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_transformer = TfidfVectorizer()
#Clean the text and remove stop words
train_data = train_data.apply(lambda x : clean_text(x))
#Fit the model and transform based of TFIDF vectorization
tfidf_vectors = tfidf_transformer.fit(train_data).transform(train_data)

In [11]:
tfidf_vectors.shape

(774, 6226)

In [12]:
#Use pipeline to classify documents based on the attributes and find the accuracy score with respect to data_y
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
predict_tf_idf = Pipeline(steps=[('tfidf',tfidf_transformer),
                         ('xgboost', XGBClassifier(objective='binary:logistic'))])
#cross validate the prediction of pipeline with test column
scores = cross_val_score(predict_tf_idf, train_data, test_data, cv=5)
print('Accuracy for Tf-Idf & XGBoost Classifier using description : ', scores.mean())

Accuracy for Tf-Idf & XGBoost Classifier using description :  0.9586761625471303


In [15]:
df = pd.read_csv("fake_job_postings.csv",engine='python',error_bad_lines=False)

In [16]:
df[df['fraudulent']==1].shape

(866, 18)

In [17]:
dat = {'text': []}
data = pd.DataFrame(data=dat)

In [18]:
df.title.fillna('', inplace=True)
df.company_profile.fillna('', inplace=True)
df.description.fillna('', inplace=True)
df.requirements.fillna('', inplace=True)

data['text'] = df['title'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements']
data
data['fraudulent'] = df['fraudulent']
data[data['fraudulent']==1].shape

(866, 2)

In [19]:
from nltk.corpus import stopwords
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdrsh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdrsh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdrsh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [20]:
stemmer = SnowballStemmer("english")
lem=WordNetLemmatizer()
data['text'] = data['text'].map(lambda x: re.sub(r"\W", " ",str(x)))
data['text'] = data['text'].map(lambda x: re.sub(r'[,\.!?]', " ",str(x)))
data['text'] = data['text'].map(lambda x: re.sub(r" \d+", " ",str(x)))
data['text'] = data['text'].map(lambda x: word_tokenize(x.lower()))
data['text'] = data['text'].map(lambda x: ' '.join(x))
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
data['lemm'] = data['text'].apply(lambda x: ' '.join([lem.lemmatize(y) for y in x.split()]))
data['stemm'] = data['lemm'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
data['text'] = data['stemm']

# data['text']=data['text'].str.lower()
# data['text']=data['text'].str.replace('\d+', '')
# data['text']=data['text'].str.replace(r"http\S+", '')
# data
# df[5690]
data

,text,fraudulent,lemm,stemm
0,market intern food52 creat groundbreak award w...,0,marketing intern food52 created groundbreaking...,market intern food52 creat groundbreak award w...
1,custom servic cloud video product second world...,0,customer service cloud video production second...,custom servic cloud video product second world...
2,commiss machineri assist cma valor servic prov...,0,commissioning machinery assistant cma valor se...,commiss machineri assist cma valor servic prov...
3,account execut washington dc passion improv qu...,0,account executive washington dc passion improv...,account execut washington dc passion improv qu...
4,bill review manag spotsourc solut llc global h...,0,bill review manager spotsource solution llc gl...,bill review manag spotsourc solut llc global h...
5,account clerk job overviewapex environment con...,0,accounting clerk job overviewapex environmenta...,account clerk job overviewapex environment con...
6,head content f found theâ fonpit agâ rose inte...,0,head content f founded theâ fonpit agâ rose in...,head content f found theâ fonpit agâ rose inte...
7,lead guest servic specialist airenvyâ mission ...,0,lead guest service specialist airenvyâ mission...,lead guest servic specialist airenvyâ mission ...
8,hp bsm sme solutions3 woman own small busi who...,0,hp bsm sme solutions3 woman owned small busine...,hp bsm sme solutions3 woman own small busi who...
9,custom servic associ part time novitex enterpr...,0,customer service associate part time novitex e...,custom servic associ part time novitex enterpr...


In [21]:
data=data.drop(columns=['lemm', 'stemm'])

In [22]:
print(data[data['fraudulent']==1].shape)
data.dropna(inplace=True)
data[data['fraudulent']==1].shape
data

(866, 2)


,text,fraudulent
0,market intern food52 creat groundbreak award w...,0
1,custom servic cloud video product second world...,0
2,commiss machineri assist cma valor servic prov...,0
3,account execut washington dc passion improv qu...,0
4,bill review manag spotsourc solut llc global h...,0
5,account clerk job overviewapex environment con...,0
6,head content f found theâ fonpit agâ rose inte...,0
7,lead guest servic specialist airenvyâ mission ...,0
8,hp bsm sme solutions3 woman own small busi who...,0
9,custom servic associ part time novitex enterpr...,0


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')
X=vectorizer.fit_transform(data['text'])

In [32]:
def fitPredict(models_dict,X_train,y_train,X_test,y_test):
  acc={}
  spec={}
  sens={}
  model_save=None
  i=0
  #models_dict holds name and its function , like {'LR':LogsticRegression()}
  for name,model in models_dict.items():
    try:
      #fitting model on the train data
      model.fit(X_train,y_train)
      joblib.dump(model, name+'.pkl')
      #predicting model on X_test 
      model_save=model
      y_pred=model.predict(X_test)
      print('*'*100)
      print('Model:',name)
      #printing Confusion matrix an dAccuracy of each model
      print("Confusion Matrix: ")
      print(confusion_matrix(y_test, y_pred))
      print("Accuracy : ",accuracy_score(y_test,y_pred)*100)

      #mainitaing a dictionary of results of accuracy of each model to use it to plot in one systyem
      acc[name]=accuracy_score(y_test,y_pred)*100
      cm=confusion_matrix(y_test, y_pred)
      sens[name] = (cm[0,0]/(cm[0,0]+cm[0,1]))*100
      spec[name] = (cm[1,1]/(cm[1,0]+cm[1,1]))*100
      print("Classificarion Report : ")
      print(classification_report(y_test, y_pred))
      
    except ValueError:
      print(name,'failed')
  
  #converting dictionaries to dataframes to have them handy
  acc=pd.DataFrame.from_dict(acc,orient='index').reset_index()
  sens=pd.DataFrame.from_dict(sens,orient='index').reset_index()
  spec=pd.DataFrame.from_dict(spec,orient='index').reset_index()
  # print(accuracy)
  acc.columns=['model_name','accuracy']
  sens.columns=['model_name','sensitivity']
  spec.columns=['model_name','specificity']
  
  #plotting accuracy,sensitivity,specificity of each model
  return model_save

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.externals import joblib 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score
from sklearn.naive_bayes import MultinomialNB

In [34]:
models_dict={}
models_dict['XGB']                      = XGBClassifier()

In [35]:
Y=data.loc[:,data.columns=='fraudulent']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=1)
print(sum(y_train.fraudulent))

745


In [36]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
sm = SMOTE(random_state=12, sampling_strategy=0.5)
x_train_res, y_train_res = sm.fit_sample(X_train, y_train.fraudulent)
undersample = NearMiss(sampling_strategy=1)
x_train_final, y_train_final = undersample.fit_sample(x_train_res, y_train_res)

In [37]:
model_save=fitPredict(models_dict,X_train,y_train,X_test,y_test)

C:\Users\sdrsh\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sdrsh\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


****************************************************************************************************
Model: XGB
Confusion Matrix: 
[[2559    2]
 [  43   78]]
Accuracy :  98.3221476510067
Classificarion Report : 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2561
           1       0.97      0.64      0.78       121

    accuracy                           0.98      2682
   macro avg       0.98      0.82      0.88      2682
weighted avg       0.98      0.98      0.98      2682



In [38]:
model_save=fitPredict(models_dict,x_train_final,y_train_final,X_test,y_test)

****************************************************************************************************
Model: XGB
Confusion Matrix: 
[[2491   70]
 [  29   92]]
Accuracy :  96.30872483221476
Classificarion Report : 
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2561
           1       0.57      0.76      0.65       121

    accuracy                           0.96      2682
   macro avg       0.78      0.87      0.82      2682
weighted avg       0.97      0.96      0.97      2682



## Observation for the XGBoost algorithm using TF-IDF pipelining with Gensim vs NLTK and TF-IDF vectorization using under sampling and oversampling is


# Accuracy for XGBoost with TF-IDF Pipelining = 95.86


# Accuracy for XGboost with TF-IDF Vectorization without sampling = 98.322

# Accuracy for XGBoost with TF-IDF Vectorization with Undersampling majority class and oversampling minority class = 96.308